# Interpreting locally a AutoML model

Execute this notebook steps whne you have an automl model on the Azure Machine Learning Registry and also a Experiment Pipeline Run succeded. Why? In the next steps you will need an automl model created and a pipeline run id to get the explanatiosn and visualize it locally.

You can use Azure Machine Learning to interpret a model by using an explainer that quantifies the amount of influence each feature contribues to the predicted label. There are many common explainers, each suitable for different kinds of modeling algorithm; but the basic approach to using them is the same.

Let's start by ensuring that you have the latest version of the Azure ML SDK installed. In addition, you'll install the Azure ML Interpretability library. You can use this to interpret many typical kinds of model, even if they haven't been trained in an Azure ML experiment or registered in an Azure ML workspace.

In [ ]:
!pip install --upgrade azureml-sdk[notebooks,automl,explain]
!pip install --upgrade azureml-interpret
# Restart the kernel after installation is complete!

**Important: Now you'll need to . In Jupyter, on the Kernel menu, select Restart and Clear Output. Then, when the output from the cell above has been removed and the kernel is restarted, continue the steps below.**

### Imports 

In [ ]:
import os
import sys
import joblib
import pickle
import warnings
warnings.filterwarnings('ignore')

from azureml.core import Workspace, Experiment
from azureml.core.model import Model, Dataset
from sklearn.model_selection import StratifiedShuffleSplit
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from azureml.train.automl.runtime.automl_explain_utilities import automl_setup_model_explanations

sys.path.append(os.path.abspath("../utils"))
from workspace import get_workspace

### Initialize Workspace

In [ ]:
ws = Workspace.from_config("../notebooks-settings/config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Download  AutoML Model and its Model Explanation

In [ ]:
Model(ws, 'heart_disease_model_automl').download(".")
with open("./model_data", "rb" ) as f:
    original_model = pickle.load(f)

In [ ]:
original_model.steps

In [ ]:
client = ExplanationClient.from_run_id(ws, "heart-disease", "<automl_pipeline_step_id>")
global_explanation = client.download_model_explanation()
global_explanation = client.download_model_explanation(top_k=4)

Note: To get the **automl_pipeline_step_id** you must go to your pipeline run on heart-disease experiment. Then select AutoML step run and click on models, as the following image: 

![automl](../../docs/autoML_locally.png)

![automl](../../docs/autoML_locally_2.png)

In [ ]:
global_explanation.classes

In [ ]:
print('ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
print('ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
print('global importance rank: {}'.format(global_explanation.global_importance_rank))

### Get Dataset

In [ ]:
preprocess_dataset = ws.datasets.get("heart_disease_preprocess_dataset")

df_test = preprocess_dataset.to_pandas_dataframe()

X = df_test.drop(['target'], axis=1)
y = df_test['target']

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

### Get AutoML Model Explainer

Let's get explainer from our model from the Azure ML AutoML Explanatiosn created during the training.

In [ ]:
automl_explainer_setup_obj = automl_setup_model_explanations(original_model, X=X_train,
                                            X_test=X_test, y=y_train,
                                            task='classification')

## Featured names

In [ ]:
automl_explainer_setup_obj.engineered_feature_names

## Number of features

In [ ]:
len(automl_explainer_setup_obj.engineered_feature_names)

## Download the explainer, registered by Pipeline Register Explainer Model step

Name of the model resgitered in the before MlOps Publish Pipeline and Submit execution.

In [ ]:
Model(ws, 'heart_disease_explainer_automl').download(".")
explainer_model = joblib.load("./scoring_explainer.pkl")

In [ ]:
automl_explainer_setup_obj = automl_setup_model_explanations(original_model,
                                                            X_test=X_test, task='classification')

local_importance_values = explainer_model.explain(
    automl_explainer_setup_obj.X_test_transform)

Local importance values for X_test dataset

In [ ]:
local_importance_values

## Visualize
Visualize the explanations

In [ ]:
from interpret_community.widget import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, original_model)

### Upload model explainability

In [ ]:
experiment_name = 'heart-disease'
experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()
client = ExplanationClient.from_run(run)

In [ ]:
client.upload_model_explanation(global_explanation, comment='global explanation: all features')